In [1]:
from shapely.geometry import Point, Polygon
import pandas as pd
import geopandas as gpd
import glob, time
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.options.mode.chained_assignment = None  # default='warn'
gpd.options.use_pygeos = True

#### Acessa e padroniza dados

In [2]:
df = pd.read_csv("../data/censo_dados_basicos.csv", dtype={'Cod_setor': str})

In [3]:
df['populacao_residente_domicilios_permanentes'] = df.populacao_residente_domicilios_permanentes.fillna(0)

In [4]:
%%time
files = glob.glob("../data/setores_censitarios_shp/*/*.shp")
gdf = pd.concat([gpd.read_file(file) for file in files])

CPU times: user 22.5 s, sys: 892 ms, total: 23.4 s
Wall time: 23.6 s


In [5]:
# Applies an 0 buffer to fix invalidity issues
gdf['geometry'] = gdf.geometry.buffer(0)

In [6]:
gdf = gdf.merge(df, left_on='CD_GEOCODI', right_on='Cod_setor', how='left')

In [9]:
%%time
spatial_index = gdf.sindex

CPU times: user 21 µs, sys: 0 ns, total: 21 µs
Wall time: 31.7 µs


#### Computa áreas

In [10]:
class inputPoint:
    
    def __init__(self, lat, lon, tracts):
        
        self.lat = lat
        
        self.lon = lon
        
        self.point = Point(lon, lat)
        
        self.counted_population = None
        
        self.find_census_tract(tracts)
                
        
    def find_census_tract(self, tracts):
        
        my_tract = tracts [ tracts.geometry.contains(self.point)].reset_index()
        
        if my_tract.shape[0] == 1:
        
            self.tract_id = my_tract.loc[0, 'CD_GEOCODI']
            self.tract_polygon = my_tract.loc[0, 'geometry']
            
        elif my_tract.shape[0] == 0:
            
            raise ValueError("Couldn't find a valid census tract")

        else:
            
            raise ValueError("How on earth can you be on more than one tract at the same time?")   

In [11]:
%%time
casaSaoPaulo = inputPoint(-23.5318,-46.6598, gdf [ gdf.Nome_do_municipio=="SÃO PAULO"])
casaPontaGrossa = inputPoint(-25.0806,-50.1921, gdf [ gdf.Nome_do_municipio=="PONTA GROSSA"])

CPU times: user 387 ms, sys: 33 µs, total: 387 ms
Wall time: 385 ms


In [12]:
def grab_neighbors(tracts, all_tracts, target, computed_tracts=None, total_people=0):

    
    # Atribui um valor padrão para computed_tracts
    if computed_tracts is None:
        computed_tracts = [ ]
    
    # Enquanto a quantidade de gente não for o bastante...
    if total_people > target:
        return computed_tracts
    
    # Adiciona a população dos setores selecionados
    for index, row in tracts.iterrows():
        
        total_people += row.populacao_residente_domicilios_permanentes
        computed_tracts.append(row.CD_GEOCODI)

        if total_people >= target:
            break

    # Chama a função com o setor e os vizinhos que não tenham sido computados ainda
    next_tracts = all_tracts[ (~all_tracts.geometry.disjoint(tracts.unary_union) ) & (~all_tracts.CD_GEOCODI.isin(computed_tracts)) ]
    
    
    return grab_neighbors(tracts = next_tracts, 
                   all_tracts = all_tracts, 
                   target = target,
                   computed_tracts = computed_tracts,
                   total_people = total_people)

In [13]:
def random_walk(tracts, all_tracts, target, computed_tracts=None, total_people=0):
    
    # Atribui um valor padrão para computed_tracts
    if computed_tracts is None:
        computed_tracts = [ ]
    
    # Base case
    if total_people > target:
        return computed_tracts
    
    # Se assegura de que estamos lidando com apenas um setor
    if tracts.shape[0] != 1:
        raise ValueError("random_walk method supports only one tract at a time")
    
    # Padroniza o índice dos itens
    tracts = tracts.reset_index()
    
    # Adiciona população e marca o setor como computado
    if tracts.loc[0, 'CD_GEOCODI'] not in computed_tracts:
        
        total_people += tracts.loc[0, 'populacao_residente_domicilios_permanentes']
        computed_tracts.append(tracts.loc[0, 'CD_GEOCODI'])
    
    # Pega os vizinhos
    neighbors = all_tracts[ (~all_tracts.geometry.disjoint(tracts.unary_union) ) ] #& (~all_tracts.CD_GEOCODI.isin(computed_tracts)) ]
        
    # Seleciona um vizinho de forma aleatória, desde que ele já não tenha sido selecionado
    tracts = neighbors.sample(n=1)
    
    # Chama a função novamente
    return random_walk(
                tracts = tracts,
                all_tracts = all_tracts,
                target = target,
                computed_tracts = computed_tracts,
                total_people = total_people)

In [14]:
def grab_by_radius(input_point, all_tracts, target, computed_tracts=None, total_people=0, radius=0):
        
    # Atribui um valor padrão para computed_tracts
    if computed_tracts is None:
        computed_tracts = [ ]
        
    # Base case
    if total_people > target:
        print(f"Total people: {total_people}")
        return computed_tracts        
        
    # Se o raio for diferente de 0, faz o buffer
    # Note que o raio do buffer é calculado na unidade de graus lat/lon
    if radius != 0:
        polygon = input_point.point.buffer(radius)
        
    else:
        polygon = input_point.point
                
    # Verifica quais são os setores que cruzam o raio
    # (Se não houver raio, o setor sempre cruzará a si mesmo)
    intersections = all_tracts [ all_tracts.geometry.intersects(polygon) ]
        
    for index, row in intersections.iterrows():
        
        total_people += row.populacao_residente_domicilios_permanentes
        computed_tracts.append(row.CD_GEOCODI)

        if total_people >= target:
            break
              
    # Chamada recursiva com raio 50% maior
    if radius == 0:
        radius = 0.001
        
    return grab_by_radius(input_point = input_point,
                          all_tracts = all_tracts [ ~all_tracts.CD_GEOCODI.isin(computed_tracts) ],
                          target = target,
                          computed_tracts = computed_tracts,
                          total_people = total_people,
                          radius = radius * 1.5)

In [41]:
def grab_by_radius_proportional(input_point, all_tracts, spatial_index, target):
        
    total_people = 0
    radius = 0
    
    # Enquanto o alvo não for atingido, seguimos crescendo o raio em 50% a cada iteração
    while total_people < target:
                
        total_people = 0
        
        if radius != 0:
                        
            polygon = input_point.point.buffer(radius)
            
        else:
            polygon = input_point.point
            
        # Pre-seleciona os polígonos passíveis de interseção
        # https://geoffboeing.com/2016/10/r-tree-spatial-index-python/
        
        checkpoint = time.time()
        
        possible_matches_index = list(spatial_index.intersection(polygon.bounds))
        
        print(f"Rtree possible matches: {len(possible_matches_index)}")

        possible_matches = all_tracts.iloc[possible_matches_index]
                
        # Transforma em polígono para computar a interseção mais rápido usando os spatial index do geopandas
        selected_tracts = possible_matches [ possible_matches.geometry.intersects(polygon) ]

        print(f"Intersecting tracts: {selected_tracts.shape[0]}")
        print(f"Intersection time: {round(time.time() - checkpoint, 4)}")
        
        
        def grab_pop(population, tract, polygon):
                        
            intersection = tract.intersection(polygon)
            
            intersection_percentage = intersection.area / tract.area 
    
            pop_to_add = population * intersection_percentage
        
            return pop_to_add
        
        checkpoint = time.time()

        # Aplica a função acima de forma vetorizado, usando npArrays (df.col.values)
        selected_tracts['pop_to_add'] = grab_pop(selected_tracts.populacao_residente_domicilios_permanentes.values,
                                                 selected_tracts.geometry.values,
                                                 polygon)
        

        total_people = round(selected_tracts.pop_to_add.sum())
    
                
        print(f"Vector time: {round(time.time() - checkpoint, 4)}")
        print(f"Total people: {total_people}")
        
        if radius == 0:
            radius = 0.001
            
        else:
            radius = radius * 1.5
        
        print()   
    
    print()
    print("Now fine tuning!")
    print()

            
    # Uma vez que tenha passado, verificamos se está na margem de tolerância.
    # Se não estiver, faz ajuste fino
    fine_tune = .5
    
    while not (total_people < target * 1.05 and total_people > target * .95):
        
        
        # Se estiver acima de 105% do alvo, decresce raio na proporção do fine_tune
        if total_people > target * 1.05:
            
            total_people = 0
            
            radius = radius - radius * fine_tune
                        
            polygon = input_point.point.buffer(radius)
                        
            # Pre-seleciona os polígonos passíveis de interseção
            # https://geoffboeing.com/2016/10/r-tree-spatial-index-python/

            checkpoint = time.time()

            possible_matches_index = list(spatial_index.intersection(polygon.bounds))
            possible_matches = all_tracts.iloc[possible_matches_index]
                        
            selected_tracts = possible_matches [ possible_matches.geometry.intersects(polygon) ]
            
            print(f"Intersecting tracts: {selected_tracts.shape[0]}")
            print(f"Intersection time: {round(time.time() - checkpoint, 4)}")

            checkpoint = time.time()
        
            selected_tracts['pop_to_add'] = grab_pop(selected_tracts.populacao_residente_domicilios_permanentes.values,
                                                 selected_tracts.geometry.values,
                                                 polygon)

            total_people = round(selected_tracts.pop_to_add.sum())

            print(f"Vector time: {round(time.time() - checkpoint, 4)}")

        
        # Se estiver abaixo, acresce na proporção do fine_tune
        elif total_people < target * .95:
        
            total_people = 0
        
            radius = radius *  (1 + fine_tune)
                        
            polygon = input_point.point.buffer(radius)
                        
            checkpoint = time.time()    
                
            # Pre-seleciona os polígonos passíveis de interseção
            # https://geoffboeing.com/2016/10/r-tree-spatial-index-python/
            possible_matches_index = list(spatial_index.intersection(polygon.bounds))
            possible_matches = all_tracts.iloc[possible_matches_index]
            
            selected_tracts = possible_matches [ possible_matches.geometry.intersects(polygon) ]
            
            print(f"Intersecting tracts: {selected_tracts.shape[0]}")
            print(f"Intersection time: {round(time.time() - checkpoint, 4)}")
      
            checkpoint = time.time()
        
            selected_tracts['pop_to_add'] = grab_pop(selected_tracts.populacao_residente_domicilios_permanentes.values,
                                                 selected_tracts.geometry.values,
                                                 polygon)
    
            total_people = round(selected_tracts.pop_to_add.sum())

            print(f"Vector time: {round(time.time() - checkpoint, 4)}")
            print(f"Total people: {total_people}")


                
        print(f"Total people: {total_people}")
        print()
        
        # Ajusta a fração
        fine_tune = fine_tune / 2
        
            
    print(f"Total people: {total_people}")
    return polygon

In [42]:
def fill(input_point, tracts, all_tracts, target, f=grab_neighbors):
    
    if f == grab_neighbors:
        return f(tracts, all_tracts, target)
    
    elif f == grab_by_radius:
        return f(input_point, all_tracts, target)
        
    elif f == grab_by_radius_proportional:
                        
        spatial_index = all_tracts.sindex
        return f(input_point, all_tracts, spatial_index, target)

In [45]:
%%time
to_highlight = fill(input_point = casaSaoPaulo,
                     tracts = gdf [gdf.CD_GEOCODI == casaSaoPaulo.tract_id] , 
                     all_tracts = gdf [ gdf.NM_MUNICIP == "SÃO PAULO" ],
                     target = 1000000,
                     f = grab_by_radius_proportional)

Rtree possible matches: 2
Intersecting tracts: 1
Intersection time: 0.0128
Vector time: 0.0009
Total people: 0.0

Rtree possible matches: 10
Intersecting tracts: 4
Intersection time: 0.0027
Vector time: 0.0012
Total people: 1150.0

Rtree possible matches: 18
Intersecting tracts: 10
Intersection time: 0.0028
Vector time: 0.0016
Total people: 2426.0

Rtree possible matches: 23
Intersecting tracts: 17
Intersection time: 0.0029
Vector time: 0.0021
Total people: 4598.0

Rtree possible matches: 35
Intersecting tracts: 29
Intersection time: 0.0029
Vector time: 0.0026
Total people: 9390.0

Rtree possible matches: 66
Intersecting tracts: 50
Intersection time: 0.0033
Vector time: 0.0033
Total people: 20331.0

Rtree possible matches: 131
Intersecting tracts: 104
Intersection time: 0.004
Vector time: 0.0057
Total people: 42545.0

Rtree possible matches: 242
Intersecting tracts: 203
Intersection time: 0.0058
Vector time: 0.0103
Total people: 84690.0

Rtree possible matches: 429
Intersecting tracts:

In [25]:
for item in invalid.geometry:
    display(item)

NameError: name 'invalid' is not defined

In [ ]:
gpd